<hr style="border:2px solid #0176DE"> </hr>
<center><h1 style="color:#173F8A;"> EMI 2024 - CMM Chile</h1></center>
<hr style="border:2px solid #0176DE"> </hr>
<h3 style="color:#173F8A;text-align:right;"> Profesores: &nbsp;Nicolás Barnafi<br>Manuel A. Sánchez<br></h3>

<h3 style="color:#03122E;text-align:right;"> 
    Centro de Modelamiento Matematico <br> 
    Instituto de Ingenieria Matematica y Computacional - IMC UC<br>  
</h3>

<hr style="border:2px solid #03122E"> </hr>
<center><h1 style="color:#173F8A;"> Modulo 3: MinRes, GMRES - NGSolve</h1></center> 
<hr style="border:2px solid #03122E"> </hr>

<!-- Palette colors UC:
Primaria: 
celeste:#0176DE, azul #173F8A, azul oscuro: #03122E, amarillo: #FEC60D, amarillo oscuro: #E3AE00 
Secundaria
gris oscuro: #707070
-->

In [269]:
from ngsolve import *
from ngsolve.webgui import Draw
from ngsolve.la import EigenValues_Preconditioner
import matplotlib.pyplot as plt

In [270]:
# help(solvers.PreconditionedRichardson)
# help(solvers.CG)
# help(solvers.GMRes)
# help(solvers.MinRes)
# help(solvers.QMR)
# help(krylovspace.CG)
# help(krylovspace.RichardsonSolver)
# help(krylovspace.GMRes)
# help(krylovspace.MinRes)
# help(krylovspace.QMR)

## TEST 1: Model Problem the $L^2$ projection

We will test the algorithm with the solution of $L^2$ projection onto the space of continuous piecewise linear functions in $(0,1)^2$ of a function
$$ f= \sin(\pi(x-y)) $$ 

In [271]:
def SetL2Proj(mesh, fun=1, p=1, condense=False):
    # H1-conforming Finite Element Space
    fes = H1(mesh, order=p)
    # Trial and Test Functions
    u, v = fes.TnT()
    # Bilinear form 
    a = BilinearForm(fes, symmetric=True, condense=condense)
    a += (u*v)*dx 
    # Linear form
    b = LinearForm(fun*v*dx)
    # Gridfunction
    gfu = GridFunction(fes)
    return fes, a, b, gfu

### Iterative method : Iterative method solver (no preconditioner) 

In [272]:
def SolveL2ProjIterative(a, b, gfu, fes, itermethod='CG', condense=False, compute_condnum=False):
    a.Assemble()
    b.Assemble()
    preI = Projector(mask=fes.FreeDofs(), range=True)
    # Solve
    if itermethod=='Richardson':
        lams = EigenValues_Preconditioner(mat=a.mat, pre=preI)
        alpha=2.0/(min(lams)+max(lams))
        gfu.vec.data = solvers.PreconditionedRichardson(a =a, 
                      rhs = b.vec, 
                      pre=None, freedofs=fes.FreeDofs(),
                      tol=1e-8,
                      maxit=10000, dampfactor=alpha,
                      printing='\r')
    elif itermethod=='CG':
        solvers.CG(mat =a.mat, 
                      rhs = b.vec, 
                      pre=None, freedofs=fes.FreeDofs(), 
                      sol =gfu.vec, 
                      tol=1e-8,
                      maxsteps=10000, 
                      printrates='\r')
    elif itermethod=='GMRes':
        solvers.GMRes(A =a.mat, 
                      b = b.vec, 
                      pre=None, freedofs=fes.FreeDofs(), 
                      x =gfu.vec, 
                      tol=1e-8,
                      maxsteps=10000, 
                      printrates='\r')
    elif itermethod=='MinRes':
        solvers.MinRes(mat =a.mat, 
                      rhs = b.vec, 
                      pre=preI, #freedofs=fes.FreeDofs(), 
                      sol =gfu.vec, 
                      tol=1e-8,
                      maxsteps=10000, 
                      printrates='\r')
    elif itermethod=='QMR':
        solvers.QMR(mat =a.mat, 
                      rhs = b.vec, 
                      fdofs=fes.FreeDofs(), 
                      sol =gfu.vec, 
                      tol=1e-8,
                      maxsteps=10000, 
                      printrates='\r')
    if compute_condnum is False:
        return gfu, ()
    else:
        lams = EigenValues_Preconditioner(mat=a.mat, pre=preI)
        kappa = max(lams)/min(lams)
        return gfu, kappa

### Test 1 parameters

In [273]:
# parametros
paramt1 = dict()
paramt1['h'] = 0.05
paramt1['f'] = sin(2*pi*(x-y))*cos(2*pi*(x+y))
paramt1['p'] = 1
paramt1['levels'] = 6 # <=8 for p=1
paramt1['h0'] = 0.05
paramt1['condense'] = False

In [274]:
def SolveL2ProjIterative_levels(h, levels, itermethod='CG', condense=False, compute_condnum=False):
    mesh = Mesh(unit_square.GenerateMesh(maxh=h))
    fes, a, b, gfu = SetL2Proj(mesh, p=paramt1['p'], fun=paramt1['f'], condense=condense)
    tiempo = []
    steps = []
    for l in range(levels):
        if l > 0: mesh.Refine()
        fes.Update()
        gfu.Update()
        with TaskManager():
            start = time()
            gfu, invsteps = SolveL2ProjIterative(a, b, gfu, fes, itermethod, condense=condense, compute_condnum=compute_condnum)
            print (f"ndof = {fes.ndof}, time = {time()-start}, steps = {invsteps}")
            tiempo.append ( (fes.ndof, time()-start ))
        steps.append ( (fes.ndof, invsteps ))
    return gfu, tiempo, steps
_, time_cg, steps = SolveL2ProjIterative_levels(h=paramt1['h0'], levels=paramt1['levels'], itermethod='Richardson', condense=paramt1['condense'], compute_condnum=False)

it = 0  ||res||_2 = 0.022213823280636057
it = 1  ||res||_2 = 0.005128275912126208
it = 2  ||res||_2 = 0.0017500547123072737
it = 3  ||res||_2 = 0.0008610643659458512
it = 4  ||res||_2 = 0.0005835134643350262
it = 5  ||res||_2 = 0.0004697057501567159
it = 6  ||res||_2 = 0.0004029649134093073
it = 7  ||res||_2 = 0.00035375081717005857
it = 8  ||res||_2 = 0.000313653225579223
it = 9  ||res||_2 = 0.0002795452588444037
it = 10  ||res||_2 = 0.00024989671930315257
it = 11  ||res||_2 = 0.00022380313820933166
it = 12  ||res||_2 = 0.00020066400677487483
it = 13  ||res||_2 = 0.00018004754354582764
it = 14  ||res||_2 = 0.0001616237565606814
it = 15  ||res||_2 = 0.00014512816693390017
it = 16  ||res||_2 = 0.00013034108068606353
it = 17  ||res||_2 = 0.00011707524958935411
it = 18  ||res||_2 = 0.00010516821418848931
it = 19  ||res||_2 = 9.44773245924714e-05
it = 20  ||res||_2 = 8.487632992653998e-05
it = 21  ||res||_2 = 7.625291307271298e-05
it = 22  ||res||_2 = 6.850681533507113e-05
it = 23  ||res||

it = 0  ||res||_2 = 0.006015000388374354
it = 1  ||res||_2 = 0.0009929570807251894
it = 2  ||res||_2 = 0.0005220986906591933
it = 3  ||res||_2 = 0.0003984831388373066
it = 4  ||res||_2 = 0.0003347961293328034
it = 5  ||res||_2 = 0.00028980589930992803
it = 6  ||res||_2 = 0.00025435745879417445
it = 7  ||res||_2 = 0.0002251724867103666
it = 8  ||res||_2 = 0.00020056395860586343
it = 9  ||res||_2 = 0.0001794804347163823
it = 10  ||res||_2 = 0.0001612016397368852
it = 11  ||res||_2 = 0.00014520711816076804
it = 12  ||res||_2 = 0.00013110721607175792
it = 13  ||res||_2 = 0.00011860234124215934
it = 14  ||res||_2 = 0.00010745712949729066
it = 15  ||res||_2 = 9.748325683266966e-05
it = 16  ||res||_2 = 8.852759994018647e-05
it = 17  ||res||_2 = 8.04638622949747e-05
it = 18  ||res||_2 = 7.318652567283974e-05
it = 19  ||res||_2 = 6.66064050326532e-05
it = 20  ||res||_2 = 6.064733296732649e-05
it = 21  ||res||_2 = 5.524365386322061e-05
it = 22  ||res||_2 = 5.033830682179131e-05
it = 23  ||res||_

it = 0  ||res||_2 = 0.0015146582226189058
it = 1  ||res||_2 = 0.0002601384628239488
it = 2  ||res||_2 = 0.00012744586431613162
it = 3  ||res||_2 = 9.604893782334374e-05
it = 4  ||res||_2 = 8.067563818140916e-05
it = 5  ||res||_2 = 6.967908009054862e-05
it = 6  ||res||_2 = 6.0812053297753465e-05
it = 7  ||res||_2 = 5.338737881625649e-05
it = 8  ||res||_2 = 4.707330190117497e-05
it = 9  ||res||_2 = 4.165610790890744e-05
it = 10  ||res||_2 = 3.697858294912466e-05
it = 11  ||res||_2 = 3.291830311758581e-05
it = 12  ||res||_2 = 2.937734069821493e-05
it = 13  ||res||_2 = 2.6276192098601603e-05
it = 14  ||res||_2 = 2.354968724129297e-05
it = 15  ||res||_2 = 2.1144019113818204e-05
it = 16  ||res||_2 = 1.901449313268155e-05
it = 17  ||res||_2 = 1.7123779235320562e-05
it = 18  ||res||_2 = 1.5440534130067342e-05
it = 19  ||res||_2 = 1.393830522786641e-05
it = 20  ||res||_2 = 1.259465332247389e-05
it = 21  ||res||_2 = 1.139044726302e-05
it = 22  ||res||_2 = 1.0309294882688838e-05
it = 23  ||res||_

it = 39  ||res||_2 = 9.021062310186831e-07
it = 40  ||res||_2 = 8.192015902904856e-07
it = 41  ||res||_2 = 7.440365932251428e-07
it = 42  ||res||_2 = 6.758698188900233e-07
it = 43  ||res||_2 = 6.140339300544323e-07
it = 44  ||res||_2 = 5.579277622104952e-07
it = 45  ||res||_2 = 5.070093425969325e-07
it = 46  ||res||_2 = 4.6078971584605876e-07
it = 47  ||res||_2 = 4.1882747145068536e-07
it = 48  ||res||_2 = 3.807238837047651e-07
it = 49  ||res||_2 = 3.461185876766583e-07
it = 50  ||res||_2 = 3.146857255851608e-07
it = 51  ||res||_2 = 2.8613050703773804e-07
it = 52  ||res||_2 = 2.601861342589426e-07
it = 53  ||res||_2 = 2.3661104993017436e-07
it = 54  ||res||_2 = 2.1518647077943455e-07
it = 55  ||res||_2 = 1.9571417476472522e-07
it = 56  ||res||_2 = 1.7801451372094458e-07
it = 57  ||res||_2 = 1.6192462679714097e-07
it = 58  ||res||_2 = 1.4729683298920036e-07
it = 59  ||res||_2 = 1.3399718364702553e-07
it = 60  ||res||_2 = 1.219041580667742e-07
it = 61  ||res||_2 = 1.1090748721910861e-07


## TEST 2: Model problem the Poisson equation
We will test the algorithm in this notebook with the solution of the following Poisson problem

\begin{equation}
-\Delta u = 1, \quad \text{in } (0,1)^{2}, \qquad u=0, \quad \text{sobre } x=0 \text{ e }y=0.
\end{equation}

In [3]:
def SetPoisson(mesh, f=1, p=1, dirichlet_bndry=None, condense=False):
    # H1-conforming Finite Element Space
    fes = H1(mesh, order=p, dirichlet=dirichlet_bndry)
    # Trial and Test Functions
    u, v = fes.TnT()
    # Bilinear form 
    a = BilinearForm(fes, symmetric=True, condense=condense)
    a += (grad(u)*grad(v))*dx 
    # Linear form
    b = LinearForm(f*v*dx)
    # Gridfunction
    gfu = GridFunction(fes)
    return fes, a, b, gfu

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

### Iterative method : Iterative method solver (no preconditioner) 

In [262]:
def SolvePoissonIterative(a, b, gfu, fes, itermethod='CG', condense=False, compute_condnum=False):
    a.Assemble()
    b.Assemble()
    preI = Projector(mask=fes.FreeDofs(), range=True)
    # Solve
    if itermethod=='Richardson':
        lams = EigenValues_Preconditioner(mat=a.mat, pre=preI)
        alpha=2.0/(min(lams)+max(lams))
        gfu.vec.data = solvers.PreconditionedRichardson(a =a, 
                      rhs = b.vec, 
                      pre=None, freedofs=fes.FreeDofs(),
                      tol=1e-8,
                      maxit=10000, dampfactor=alpha,
                      printing='\r')
    elif itermethod=='CG':
        solvers.CG(mat =a.mat, 
                      rhs = b.vec, 
                      pre=None, freedofs=fes.FreeDofs(), 
                      sol =gfu.vec, 
                      tol=1e-8,
                      maxsteps=10000, 
                      printrates='\r')
    elif itermethod=='GMRes':
        solvers.GMRes(A =a.mat, 
                      b = b.vec, 
                      pre=None, freedofs=fes.FreeDofs(), 
                      x =gfu.vec, 
                      tol=1e-8,
                      maxsteps=10000,
                      printrates='\r')
    elif itermethod=='MinRes':
        solvers.MinRes(mat =a.mat, 
                      rhs = b.vec, 
                      pre=preI, #freedofs=fes.FreeDofs(), 
                      sol =gfu.vec, 
                      tol=1e-8,
                      maxsteps=10000, 
                      printrates='\r')
    elif itermethod=='QMR':
        solvers.QMR(mat =a.mat, 
                      rhs = b.vec, 
                      fdofs=fes.FreeDofs(), 
                      sol =gfu.vec, 
                      tol=1e-8,
                      maxsteps=10000, 
                      printrates='\r')
    if compute_condnum is False:
        return gfu, () 
    else:
        lams = EigenValues_Preconditioner(mat=a.mat, pre=preI)
        kappa = max(lams)/min(lams)
        return gfu,kappa

In [263]:
class iter_counter(object): # for CG
    def __init__(self):
        self.niter = 0
    def __call__(self, iterations, residual):
#         self.niter += 1
        self.iterations=iterations
        self.residual = residual

### Test 2 parameters

In [264]:
# parameters
paramt2 = dict()
paramt2['h'] = 0.05
paramt2['f'] = 1
paramt2['p'] = 1 # p = 6 try h0=0.25
paramt2['h0'] = 0.05
paramt2['levels'] = 4 # <=6 if p=1
paramt2['condense'] = False # True does not work

## Iterative Methods Richardson, CG, MinRes, GMRES, QMR (no preconditioner)

In [265]:
def SolvePoissonIterative_levels(h, levels, itermethod='CG', condense=False, compute_condnum=False):
    mesh = Mesh(unit_square.GenerateMesh(maxh=h))
    fes, a, b, gfu = SetPoisson(mesh, p=paramt2['p'], dirichlet_bndry='bottom|left', condense=condense)
    tiempo = []
    steps = []
    for l in range(levels):
        if l > 0: mesh.Refine()
        fes.Update()
        gfu.Update()
        with TaskManager():
            start = time()
            gfu, invsteps = SolvePoissonIterative(a, b, gfu, fes, itermethod, condense=condense, compute_condnum=compute_condnum)
            print (f"ndof = {fes.ndof}, time = {time()-start}, steps = {invsteps}")
            tiempo.append ( (fes.ndof, time()-start ))
        steps.append ( (fes.ndof, invsteps ))
    return gfu, tiempo, steps

_, time_cg, steps = SolvePoissonIterative_levels(h=paramt2['h0'], levels=paramt2['levels'], itermethod='QMR', condense=paramt2['condense'], compute_condnum=False)

QMR converged in 84 iterations to residual 3.4922608467786384e-10
ndof = 508, time = 0.1439657211303711, steps = ()
QMR converged in 219 iterations to residual 2.0944423370591147e-10
ndof = 1949, time = 0.28226590156555176, steps = ()
QMR converged in 434 iterations to residual 1.14041827056971e-10
ndof = 7633, time = 0.7309849262237549, steps = ()
QMR converged in 866 iterations to residual 5.823262125225393e-11
ndof = 30209, time = 2.9644551277160645, steps = ()


### CGSolver from ngsolve.la vs. krylovspace.CG

In [275]:
help(CGSolver)
help(krylovspace.CG)

Help on built-in function CGSolver in module ngsolve.la:

CGSolver(...) method of builtins.PyCapsule instance
    CGSolver(mat: BaseMatrix, pre: BaseMatrix, complex: bool = False, printrates: bool = True, precision: float = 1e-08, maxsteps: int = 200, conjugate: bool = False) -> ngsolve.la.KrylovSpaceSolver
    
    
    A CG Solver.
    
    Parameters:
    
    mat : ngsolve.la.BaseMatrix
      input matrix 
    
    pre : ngsolve.la.BaseMatrix
      input preconditioner matrix
    
    complex : bool
      input complex, if not set it is deduced from matrix type
    
    printrates : bool
      input printrates
    
    precision : float
      input requested precision. CGSolver stops if precision is reached.
    
    maxsteps : int
      input maximal steps. CGSolver stops after this steps.

